In [1]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
import synthesizer,transformer,sampler,conditional,models

In [2]:
INPUT_FILE_PATH = "~/Downloads/toy_data.csv" #"~/Downloads/hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.csv"
SAVE_PROCESSED_ORIGNAL_DATA_PATH = "/Users/sahanalva/Counterfactual Research/toy_data_full_200_original_processed_data.csv"
SAVE_MODEL_PATH = "/Users/sahanalva/Counterfactual Research/toy_data_full_200.pt"

In [3]:
df = pd.read_csv(INPUT_FILE_PATH) 

In [4]:
required_cols = ['loan_type_name','loan_purpose_name','agency_abbr','property_type_name','state_name','preapproval_name', 'purchaser_type_name','loan_amount_000s', 'tract_to_msamd_income', 'population','number_of_1_to_4_family_units','minority_population','applicant_income_000s','hud_median_family_income','number_of_owner_occupied_units']
conditional_cols = None

In [5]:
df = df[required_cols]

In [6]:
nunique = df.apply(pd.Series.nunique)
discrete_columns = nunique[nunique < 100].index

In [7]:
numeric_columns = df.select_dtypes(include=['number']).columns
object_columns = df.select_dtypes(include=['object']).columns
df[numeric_columns] = df[numeric_columns].fillna(0)
df[object_columns] = df[object_columns].fillna("NA")

df.isna().any()

loan_type_name                    False
loan_purpose_name                 False
agency_abbr                       False
property_type_name                False
state_name                        False
preapproval_name                  False
purchaser_type_name               False
loan_amount_000s                  False
tract_to_msamd_income             False
population                        False
number_of_1_to_4_family_units     False
minority_population               False
applicant_income_000s             False
hud_median_family_income          False
number_of_owner_occupied_units    False
dtype: bool

In [14]:
model = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model.fit(df, discrete_columns, conditional_cols, 200,log_frequency= False)

oss Entropy Loss tensor(0.0029, grad_fn=<DivBackward0>)
Epoch 58, Loss G: 0.8860, Loss D: -0.1948, Loss Conditional D: -0.6547
Condtional Cross Entropy Loss tensor(0., grad_fn=<DivBackward0>)
Epoch 59, Loss G: 1.0776, Loss D: -0.2054, Loss Conditional D: -0.6532
Condtional Cross Entropy Loss tensor(0.0292, grad_fn=<DivBackward0>)
Epoch 60, Loss G: 1.0554, Loss D: -0.0725, Loss Conditional D: -0.3364
Condtional Cross Entropy Loss tensor(0.3991, grad_fn=<DivBackward0>)
Epoch 61, Loss G: 1.3211, Loss D: -0.0721, Loss Conditional D: -0.0981
Condtional Cross Entropy Loss tensor(1.4075, grad_fn=<DivBackward0>)
Epoch 62, Loss G: 2.9700, Loss D: -0.0662, Loss Conditional D: -0.2402
Condtional Cross Entropy Loss tensor(0., grad_fn=<DivBackward0>)
Epoch 63, Loss G: 1.3522, Loss D: -0.0442, Loss Conditional D: -0.0620
Condtional Cross Entropy Loss tensor(0.0421, grad_fn=<DivBackward0>)
Epoch 64, Loss G: 0.9892, Loss D: -0.0404, Loss Conditional D: -0.2131
Condtional Cross Entropy Loss tensor(0., 

In [45]:
sampled = model.sample(100000,col_index= [10])

tensor(2.7603, grad_fn=<DivBackward0>)
tensor(2.7114, grad_fn=<DivBackward0>)
tensor(2.6205, grad_fn=<DivBackward0>)
tensor(2.7856, grad_fn=<DivBackward0>)
tensor(2.4013, grad_fn=<DivBackward0>)
tensor(2.6401, grad_fn=<DivBackward0>)
tensor(2.7069, grad_fn=<DivBackward0>)
tensor(2.7439, grad_fn=<DivBackward0>)
tensor(2.5895, grad_fn=<DivBackward0>)
tensor(2.4820, grad_fn=<DivBackward0>)
tensor(2.6993, grad_fn=<DivBackward0>)
tensor(2.8470, grad_fn=<DivBackward0>)
tensor(2.6763, grad_fn=<DivBackward0>)
tensor(2.8556, grad_fn=<DivBackward0>)
tensor(2.8899, grad_fn=<DivBackward0>)
tensor(2.7243, grad_fn=<DivBackward0>)
tensor(2.9200, grad_fn=<DivBackward0>)
tensor(2.7496, grad_fn=<DivBackward0>)
tensor(2.7898, grad_fn=<DivBackward0>)
tensor(2.7296, grad_fn=<DivBackward0>)
tensor(2.7455, grad_fn=<DivBackward0>)
tensor(2.7908, grad_fn=<DivBackward0>)
tensor(2.8587, grad_fn=<DivBackward0>)
tensor(2.6173, grad_fn=<DivBackward0>)
tensor(2.6669, grad_fn=<DivBackward0>)
tensor(2.6240, grad_fn=<D

In [15]:
torch.save(model, SAVE_MODEL_PATH)
df.to_csv(SAVE_MODEL_PATH, index = False)